# Terrascope data download authentication
This notebook gives examples showing how you can authenticate requests to download the products in the TerraCatalogue.

Multiple authentication methods are available and will be implemented below.
- Basic authentication
- Bearer tokens

### Set up your environment

#### Import packages

In [2]:
import requests
import base64
from pathlib import Path
import getpass
import tempfile
import ipywidgets as widgets

#### Generate password input form

In [4]:
valid_output = widgets.Valid(value=False)

def on_submit_func(button):
    button.disabled=True
    valid_output.value=True

pwd_input = widgets.Password(value='password',
                             placeholder='Enter password',
                             description='Password:',
                             disabled=False)


pwd_input.observe(on_submit_func,'value')

left_box = widgets.VBox([pwd_input])
right_box = widgets.VBox([valid_output])
widgets.HBox([left_box, right_box])

#### Set variables

In [5]:

temporary_output_dir = tempfile.TemporaryDirectory()  # Create a temporary directory to store sample download files

product_download_url = "https://services.terrascope.be/download/Sentinel5P/L3_SO2CBR_TY_V2/S5P_PAL__L3_SO2CBR_TY_20190101_V200/S5P_SO2CBR_TY_2019_SO2_V200.tif"
terrascope_username = getpass.getuser()
terrascope_password = pwd_input.value
target_dirctory = Path(temporary_output_dir.name)

### Basic Authentication

#### Encode your terrascope credentials

In [6]:
encoded_credentials = base64.b64encode(f"{terrascope_username}:{terrascope_password}".encode())  # note the : between username and password

#### Make an authenticated HTTP call to the download service

In [7]:

download_response = requests.get(url=product_download_url, headers={"Authorization": f"Basic {encoded_credentials}"})

#### Write the response content to a local file

In [8]:
with open(target_dirctory / 'filename.tif', "wb") as f:
    f.write(download_response.content)

### Bearer token authentication

#### Request credentials for the Terrascope download service based on your Terrascope account

In [10]:
authentication_response = requests.post(
    "https://sso.terrascope.be/auth/realms/terrascope/protocol/openid-connect/token",
    data={
        "grant_type": "password",
        "client_id": "public",
        "username": terrascope_username,
        "password": terrascope_password,
    }
).json()

#  Extract the necessary information from the credential request
access_token = authentication_response.get("access_token")
refresh_token = authentication_response.get("refresh token")

print(f"The access_token will be valid for {authentication_response.get('expires_in')} seconds")

The access_token will be valid for 300 seconds


#### Make download requests using the access_token
This will succeed as long as the `access_token` is valid.

When the access_token is no longer valid, you will get errors like `requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url`

In [11]:
download_response = requests.get(product_download_url, headers={"Authorization": f"Bearer {access_token}"})

#### Write the response content to a local file

In [12]:
with open(target_dirctory / 'filename.tif', "wb") as f:
    f.write(download_response.content)

#### When the access token is no longer valid, you will need to use the refresh_token to obtain a new access_token

In [13]:
refresh_response = requests.post(
    "https://sso.terrascope.be/auth/realms/terrascope/protocol/openid-connect/token",
    data={
        "grant_type": "refresh_token",
        "client_id": "public",
        "refresh_token": refresh_token
    }
)

#  Extract the necessary information from the credential refresh request
access_token = refresh_response.json().get("access_token")
refresh_token = refresh_response.json().get("refresh token")

Now you can again use the `access_token` to download data.
Repeat this cycle until you have downloaded all necessary products.

### Bearer token authentication with automatic token refresh

#### Request credentials for the Terrascope download service based on your Terrascope account

In [14]:
authentication_response = requests.post(
    "https://sso.terrascope.be/auth/realms/terrascope/protocol/openid-connect/token",
    data={
        "grant_type": "password",
        "client_id": "public",
        "username": terrascope_username,
        "password": terrascope_password,
    }
).json()

#### Loop through a necessary download links and handle token refresh when the `access_token` expires

In [ ]:
for product_url in [product_download_url]:
    for attempt in range(2):  # retry one time to be able to refresh the access token if needed
        try:
            download_response = requests.get(
              url=product_url,
              headers={'Authorization': f"Bearer {authentication_response.get('access_token')}"}
            )
            download_response.raise_for_status()  # raise an exception for non-ok HTTP status codes to get in the refresh logic
            with open(target_dirctory / product_url.split("/")[-1], "wb") as f:  # dynamically select the filename from the product_url
                f.write(download_response.content)
            break  # do not try again when the download succeeds
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 401:  # This status code indicates an issue with authentication
                authentication_response = requests.post(
                    "https://sso.terrascope.be/auth/realms/terrascope/protocol/openid-connect/token",
                    data={
                        "grant_type": "refresh_token",
                        "client_id": "public",
                        "refresh_token": authentication_response.get("refresh_token")
                    }
                ).json()
                print("refreshed access_token")
            else:
                raise(e)  # Handling other cases is out of the scope of this example

### Remove the temporary directory

In [20]:
temporary_output_dir.cleanup()